# Introduction

This notebook will demonstrate how to create a basic recommender system (RS) for books. The RS will take as input a historic set of book ratings from multiple reviewers. For a given book it then suggests what other books might be interesting. We find this in some online shops as well.

The notebook is build up as follows:
1. A little background on Recommender Systems
2. Data inspection and data preparation
3. Building the Recommender System and final exercises

This notebook is by no means complete. The purpose is to give a quick hands-on experience with building a data product.

## 1. Recommender System types

The essence of a RS is to select items an user is most likely interested in. RS exist in many different forms but we can group them into three sets:

- Collaborative filtering
- Content based filtering
- Hybrid forms

### Collaborative Filtering

Collaborative Filtering RS uses the 'wisdom of the crowd' principle and are found in two flavors: User Based and Item Based. The input of Collaborative Filtering RS is a user-item matrix with ratings of users for several items. Both RS recommend items by finding similarities between users or items respectively based on the ratings. The assumption is that if you like an item then you probably will like similar items too, or you will like what other people like who share a similar item preference.

The key here is similarity.

**Item-based** RS are easier to use because the similarities can be calculated in adcance but are of lower quality than User based models. They are mostly found in online systems shops.

**User-based** RS are more accurate but do not scale well as the complexity and required computation power increases when more users are used as reference.

### Content based filtering

Content based models focus on user profiles and item descriptions, for its recommendations. These models typically try to recommend items that are similar to what a user has liked in the past.

### Hybrid

Combinations of Collaborative Filtering and Content based filtering can be used to use the best of both worlds and to improve the quality of the RS. It seems that these types of RS are most effective of the three.

## 2. Inspect data and prepare the data

We are going the create a book recommender and for that we need user ratings. We will use a dataset from [Book-Crossing](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) but this one is already transformed into usable user-book-rating records. So the hard work has been done. ;-)

We also have to think about how we will evaluate the performance of our RS in the end and therefore how to prepare the data.

As a start we import some python libraries that we require for our prototype.

In [ ]:
# Import some important libraries
import pandas as pd
import numpy as np
#import json
%matplotlib inline

Lets read-in the data and show the 5 rows to see how the data looks like.

In [ ]:
data = pd.read_csv("data_books.csv", sep = ",", header=None,
                         names=['Reviewer', 'Book', 'Rating'])

# Let's see how the dataframe looks like
data.head()

Lets get some figures about the data:
- What is the size of the data?
- How many unique books and unique reviewers?
- How sparse is the user-item matrix?

In [ ]:
# Get the dimensions of the dataframe
dim = data.shape
print ("There are {:d} rows in this dataframe and {:d} features.".format(dim[0], dim[1]))

In [ ]:
unique_books = pd.unique(data[['Book']].values.ravel()).size
print ("{:d} unique books.".format(unique_books))

In [ ]:
unique_reviewers = pd.unique(data[['Reviewer']].values.ravel()).size
print ("{:d} unique reviewers.".format(unique_reviewers))

To see how sparse the matrix is, we calculate the fraction of non-zero cells in the matrix.

In [ ]:
# Total matrix would be
total_cells = unique_reviewers * unique_books

# How sparse is the matrix now
perc = dim[0] / float(total_cells) * 100.
print ('{:.4f}%'.format(perc))

 **Exercise**: How can we evaluate the performance of a top-N recommender system? And how should we handle the data?

>* Take out random items (rated) per user and check if the recommender will recommend that item. --> Hit Rate
* To be more precise we should take into account the recommended position --> Average Reciprocal Hit Range
* Sort groups high to low
* Pick top row for test
* Pick rest for train

Split the data and store the training and testing data as files.

```python
# This is working code, but will take about 15 minutes to run
train = pd.DataFrame()
test = pd.DataFrame()

grouped = data.groupby(['Reviewer'])

for k, g in grouped:
    # We are only interested in reviewers with more than 2 reviews
    if len(g) > 2:
        print k
        g.sort_values('Rating', ascending=False, inplace=True)
        test = test.append(g.iloc[:1])
        train = train.append(g.iloc[1:])
        
train.to_csv('train_books.csv', index=False)
test.to_csv('test_books.csv', index=False)
```

In [ ]:
train = pd.read_csv("train_books.csv", sep = ",")
test = pd.read_csv("test_books.csv", sep = ",")

In [ ]:
print ('Train: {:d}, Test: {:d}'.format(train.shape[0], test.shape[0]))

Determine the amount of reviews per book and show the top 20.

In [ ]:
# Top 20 most reviewed books 
top_books = pd.value_counts(train.Book)
top_books.head(20)

Lets show a graph of the top 1000 most reviewed books.

In [ ]:
top_books['row'] = range(1, len(top_books) + 1)
top_books[:1000].plot(x=top_books.row)

Now determine how many reviews each user has given.

In [ ]:
# Top Reviewers
top_reviewers = pd.value_counts(train.Reviewer)
top_reviewers.head(20)

**Question**: Do we see something interesting?

## 3. Building the Recommender System

### Calculate Similarities

The **Manhattan-distance** which is the sum of the absolute differences of two vectors.
$$d_{u,v} = \sum_{i=1}^{n} \left | v_{i} - u_{i} \right |$$

The **Euclidean-distance** which is the most intuitive one and calculated by pythagoras theorem.
$$d_{u,v} = \sqrt {\sum_{i=1}^{n} (v_{i} - u_{i})^2}$$

The **Pearson-correlation** which is a more complex one. It looks at how two variables changes together relatively to their individuals changes. Values range from -1 to 1.
$$\rho_{v,u} = \frac{\sum_{i=1}^{n} (v_{i} - \overline{v})( u_{i} - \overline{u})}{\sqrt{\sum_{i=1}^{n} (v_{i} - \overline{v})^{2} \sum_{i=1}^{n}( u_{i} - \overline{u})^2}}$$

The **Cosine-similarity** which measures the distance by calculating the angle from the origin. Values range from -1 to 1.
$$\cos (\theta) = \frac{v \cdot w }{\left \| v \right \|*\left \| w \right \|} = \frac{\sum_{i=1}^{n} v_{i} u_{i}}{\sqrt{\sum_{i=1}^{n} v_{i}^{2}} \sqrt{\sum_{i=1}^{n} u_{i}^{2}}}$$


For item based RS the cosine-similarity tends to be the best performing method. For user based RS the Pearson-correlation method works best. We will build both.

First we start simple by building an item based RS, using Pearson-correlation.

**Approach**:
- Choose two books
- Determine their shared reviewers
- Get for each book a list of all reviews of their shared reviewers.
- Calculate the Pearson-correlation over these reviews

In [ ]:
# 2 books 
book_1, book_2 = "Harry Potter and the Chamber of Secrets (Book 2)", "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"

Determine the shared reviewers.

In [ ]:
# Getting all the reviewers for these books
book_1_reviewers = train[train.Book == book_1].Reviewer
book_2_reviewers = train[train.Book == book_2].Reviewer

# Determine any common reviewers
common_reviewers = set(book_1_reviewers).intersection(book_2_reviewers)

print "%d people have reviewed these 2 books" % len(common_reviewers)

Get all ratings from the shared reviewers per book.

In [ ]:
# Checking the table with only the common reviewers and get the book name and rating
com_rev_book1 = train[(train.Reviewer.isin(common_reviewers)) & (train.Book == book_1)]
com_rev_book2 = train[(train.Reviewer.isin(common_reviewers)) & (train.Book == book_2)]

In [ ]:
com_rev_book1.head(10)

In [ ]:
com_rev_book2.head(10)

We see duplicate reviewers, that must be fixed to calculate the Pearson-correlation as we need two vectors of the same size.

In [ ]:
# Fix the duplicates to prevent errors when calculating Pearson-correlation.
com_rev_book1.sort_values('Reviewer')
com_rev_book1 = com_rev_book1[com_rev_book1.Reviewer.duplicated()==False]

com_rev_book1.sort_values('Reviewer')
com_rev_book1 = com_rev_book1[com_rev_book1.Reviewer.duplicated()==False]

In [ ]:
import numpy as np

In [ ]:
# The Pearson-correlation from the Numpy package
np.corrcoef(com_rev_book1.Rating, com_rev_book2.Rating)[1,0]

Now we have two books that look quit correlated.

In order to apply this calculation to a bigger set of books we need to define two functions that will help us.

1. A function that helps us to retrieve the reviews for a given book while sharing reviewers with another book
2. A function that helps us calculating the Pearson-correlation for two sets of reviews

In [ ]:
# Let's create a function that collect the reviews of our common reviewers for a specified book
def get_book_reviews(title, common_reviewers):
    mask = (train.Reviewer.isin(common_reviewers)) & (train.Book==title)
    reviews = train[mask].sort_values('Reviewer')
    reviews = reviews[reviews.Reviewer.duplicated()==False]
    return reviews

In [ ]:
def calculate_pearson_correlation(book1, book2):
    # We start by finding the common reviewers
    book_1_reviewers = train[train.Book == book1].Reviewer
    book_2_reviewers = train[train.Book == book2].Reviewer
    common_reviewers = set(book_1_reviewers).intersection(book_2_reviewers)

    # Then we look for the reviews given by common reviewers
    book_1_reviews = get_book_reviews(book1, common_reviewers)
    book_2_reviews = get_book_reviews(book2, common_reviewers)
    
    # Calculate the Pearson Correlation Score
    return np.corrcoef(book_1_reviews.Rating, book_2_reviews.Rating)[1,0]

# Print the correlation score
calculate_pearson_correlation(book_1,book_2)

**Exercise**: build a similar function that calculates the cosine-similarity between two books.

In [ ]:
def cosin_sim(v, w):
    return np.dot(v, w) / np.math.sqrt(np.dot(v, v) * np.dot(w, w))

def calculate_cosine_similarity(book1, book2):
    # We start by finding the common reviewers
    book_1_reviewers = train[train.Book == book1].Reviewer
    book_2_reviewers = train[train.Book == book2].Reviewer
    common_reviewers = set(book_1_reviewers).intersection(book_2_reviewers)

    # Then we look for the reviews given by common reviewers
    book_1_reviews = get_book_reviews(book1, common_reviewers)
    book_2_reviews = get_book_reviews(book2, common_reviewers)
    
    # Calculate the Pearson Correlation Score
    return cosin_sim(book_1_reviews.Rating, book_2_reviews.Rating)

# Print the correlation score
calculate_cosine_similarity(book_1,book_2)

## Let's expand the idea to a larger set of books

For performance reasons we will select the top most reviewed books as the calculation of similarity between all book pairs will take too much time. This is the reason why these calculations are executed seperately from the online scoring mechanism.

In [ ]:
# As I want to avoid dealing with a huge sparse matrix, I will only select the top most reviewed books for our example 

most_reviewed_books = pd.DataFrame({'count' : train.groupby(["Book"]).size()})\
                                    .reset_index().sort_values(['count'],ascending = False)

most_reviewed_books.head(20)

In [ ]:
# Getting the list of the most reviewed books

top_books = []

for i in most_reviewed_books.Book[0:20]:
    top_books.append(i)

    
# calculate the correlation for our top books
correlation_coefficient = []

for book1 in top_books:
    print "Calculating the correlations for:", book1
    for book2 in top_books:
        if book1 != book2:
            row = [book1, book2] + [calculate_pearson_correlation(book1, book2)]
            correlation_coefficient.append(row)

In [ ]:
# Let's take a look at what the table of correlation looks like
cols = ["Book_1", "Book_2", "Correlation"]
correlation_coefficient = pd.DataFrame(correlation_coefficient, columns=cols).sort_values('Correlation')
#correlation_coefficient.head(10)

In [ ]:
# Let's clean up some of the titles
correlation_coefficient= correlation_coefficient.replace(["Where the Heart Is (Oprah's Book Club (Paperback))", "Harry Potter and the Goblet of Fire (Book 4)","The Red Tent (Bestselling Backlist)", "Harry Potter and the Chamber of Secrets (Book 2)",
                                 "Angels &amp", "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))", "Harry Potter and the Prisoner of Azkaban (Book 3)"],
                                ["Where the Heart Is (Oprah's Book Club)", "Harry Potter and the Goblet of Fire", "The Red Tent", "Harry Potter and the Chamber of Secrets", "Angels & Demons",
                                 "Harry Potter and the Sorcerer's Stone", "Harry Potter and the Prisoner of Azkaban"])

In [ ]:
# Also update the top book list
top_books[6] = "Harry Potter and the Chamber of Secrets"
top_books[7] = "Angels & Demons"
top_books[9] = "Harry Potter and the Sorcerer's Stone"
top_books[10] = "Harry Potter and the Prisoner of Azkaban"
top_books[12] = "The Red Tent"
top_books[13] = "Harry Potter and the Goblet of Fire"
top_books[18] = "Where the Heart Is (Oprah's Book Club)"

In [ ]:
# Helper function to get the correlation between two books
def calc_correlation(corr, book1, book2):
    mask = (corr.Book_1==book1) & (corr.Book_2==book2)
    row = corr[mask]
    corr = row
    return corr.sum(axis=1).tolist()[0]

In [ ]:
calc_correlation(correlation_coefficient,"Harry Potter and the Sorcerer's Stone", "Harry Potter and the Chamber of Secrets")

### Check: Find similar books to "Harry Potter and the Sorcerer's Stone"

In [ ]:
# Get the sorted correlations for Harry Potter
my_book = "Harry Potter and the Sorcerer's Stone"

results = []
for b in top_books:
    if my_book!=b:
        results.append((b, calc_correlation(correlation_coefficient, my_book, b)))
sorted(results, key=lambda x: x[1], reverse=True)

**Exercise**: create a function *predict* which takes a book as input and outputs a list with other books, sorted from most correlated to lowest.

In [ ]:
def predict(my_book):
    results = []
    for other_book in top_books:
        if my_book != other_book:
            correlation = calc_correlation(correlation_coefficient, my_book, other_book)
            results.append((my_book, other_book, correlation)) 
    return pd.DataFrame.from_dict(sorted(results, key=lambda x: x[2], reverse=True))[:5] 

Now we can test for any given book, what suggestions we should consider.

In [ ]:
predict("Harry Potter and the Sorcerer's Stone")

**Question 1**: What is needed to apply the cosine-similarity function we build before, for this recommender?

**Question 2**: How could we improve the Item-based recommender system?

**Discussion**: How would you apply this in your company or job?

## References

- Part of the code is from: http://www.mickaellegal.com/blog/2014/1/30/how-to-build-a-recommender
- CF technique backgrounds are explained at http://www.hindawi.com/journals/aai/2009/421425/
- Item-Based Top-N Recommendation Algorithms (Deshpande and Karypis)
